# Fact Checking Run Benchmark Notebook
This notebook enables experimentation with contrastive knowledge assessment. You can run models including `Flan-t5`, `Bert`, and `GPT2` (among many others, see [repo](https://github.com/daniel-furman/Capstone#model-families-tested)).
The goal is to probe if factual statements are predicted at a higher probability than a given counterfactual.

<a target="_blank" href="https://colab.research.google.com/github/daniel-furman/Capstone/blob/main/notebooks/fact_checking_notebooks/fact-checking-run-main.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>


## Dependencies

In [ ]:
!git clone https://github.com/daniel-furman/Capstone.git
!pip install -r /content/Capstone/requirements.txt

## Imports

In [ ]:
import os
import glob
import torch
from argparse import Namespace
from transformers import set_seed
from datasets import load_dataset

os.chdir('/content/Capstone/src/fact_checking_scripts')
from compare_models import compare_models

## Setup

In [ ]:
# args config for running the benchmark
args = Namespace(
    model='google/bert_uncased_L-2_H-128_A-2', 
    language='en',
) 

args

In [ ]:
# ensure GPU access
if not torch.cuda.is_available():
    raise Exception("Change runtime type to include a GPU.")

In [ ]:
# set warning level
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

## Run

In [ ]:
# run the fact checking benchmark
print("Running the fact_checking benchmark...")

set_seed(42)

# load in the dataset corresponding to the input language
if (args.language.lower() == "english") or (args.language.lower() == "en"):
    dataset = load_dataset("CalibraGPT/Fact_Checking", split="English")
else:
    raise Exception("Language not supported.")

# check the input model is compatible
compatible_model_prefixes = [
    "flan",
    "t5",
    "pythia",
    "gpt",
    "opt",
    "llama",
    "roberta",
    "bert",
    "bloom",
]

model_supported = False
for model_prefix in compatible_model_prefixes:
    if model_prefix in args.model.lower():
        model_supported = True

if not model_supported:
    raise Exception("Model not supported.")

# create a config for running the pipeline
config = {
    "models": [args.model],
    "input_information": dataset,
    "verbosity": False,
}

# run the contrastive knowledge assessment function
# logs saved at './content/logging/'
score_dicts = compare_models(
    config["models"], config["input_information"], config["verbosity"]
)

# print the summary results
print(f"\nScore dict summary:\n{score_dicts[1]}")